# Capstone Project - Assignment 2 - Segmenting and Clustering - Toronto Neighborhoods

## Part 3 - Cluster the Toronto Neighborhoods

This Notebook has been created for my data science Capstone project. It will be a portfolio of my work for my Capstone journey.

## Section - Clustering Neighborhoods in Toronto

Download the relevant dependencies.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
dfPcode2 = pd.read_csv('end_part2.csv')

print('Data downloaded and read into a dataframe!')

Data downloaded and read into a dataframe!


In [3]:
dfPcode2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [4]:
dfPcode2.dtypes

PostalCode       object
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object

In [5]:
print('The dataframe has {} Boroughs and {} Postal Codes. For the purpose of this analysis the Postal Code will represent all the neighborhoods within it.'.format(
        len(dfPcode2['Borough'].unique()),
        len(dfPcode2['PostalCode'].unique())
    )
)

The dataframe has 11 Boroughs and 103 Postal Codes. For the purpose of this analysis the Postal Code will represent all the neighborhoods within it.


To make things clearer I concatenated the Postal Codes and Neighborhood columns since the postal code represents the neighborhoods due to all the neighborhood coordinates within the same postal code being the same in the table. 

In [6]:
dfPcode2['PostalCode_and_Neighborhood'] = dfPcode2['PostalCode'] +' - '+ dfPcode2['Neighborhood']
dfPcode2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,PostalCode_and_Neighborhood
0,M3A,North York,Parkwoods,43.753259,-79.329656,M3A - Parkwoods
1,M4A,North York,Victoria Village,43.725882,-79.315572,M4A - Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,"M5A - Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,"M6A - Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,M7A - Queen's Park


In [7]:
dfPcode2.drop(['PostalCode','Neighborhood'],axis=1,inplace=True)

In [8]:
dfPcode2.rename(columns={'PostalCode_and_Neighborhood':'Neighborhood'}, inplace=True)
dfPcode2.head()

,Borough,Latitude,Longitude,Neighborhood
0,North York,43.753259,-79.329656,M3A - Parkwoods
1,North York,43.725882,-79.315572,M4A - Victoria Village
2,Downtown Toronto,43.654260,-79.360636,"M5A - Harbourfront,Regent Park"
3,North York,43.718518,-79.464763,"M6A - Lawrence Heights,Lawrence Manor"
4,Queen's Park,43.662301,-79.389494,M7A - Queen's Park


In [9]:
clist=['Borough','Neighborhood','Latitude','Longitude']
dfPcode = dfPcode2[clist]
dfPcode.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,M3A - Parkwoods,43.753259,-79.329656
1,North York,M4A - Victoria Village,43.725882,-79.315572
2,Downtown Toronto,"M5A - Harbourfront,Regent Park",43.654260,-79.360636
3,North York,"M6A - Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,Queen's Park,M7A - Queen's Park,43.662301,-79.389494


Get the geographical coordinates of Toronto.

In [10]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [11]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfPcode['Latitude'], dfPcode['Longitude'], dfPcode['Borough'], dfPcode['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [13]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

#radius = 500 # define radius

#### Let's explore the neighborhoods in Toronto.

Get the neighborhood's name.

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Running function on each neighborhood in our dataframe.

In [15]:
toronto_venues = getNearbyVenues(names=dfPcode['Neighborhood'],
                                   latitudes=dfPcode['Latitude'],
                                   longitudes=dfPcode['Longitude']
                                  )

M3A - Parkwoods
M4A - Victoria Village
M5A - Harbourfront,Regent Park
M6A - Lawrence Heights,Lawrence Manor
M7A - Queen's Park
M9A - Islington Avenue
M1B - Rouge,Malvern
M3B - Don Mills North
M4B - Woodbine Gardens,Parkview Hill
M5B - Ryerson,Garden District
M6B - Glencairn
M9B - Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
M1C - Highland Creek,Rouge Hill,Port Union
M3C - Flemingdon Park,Don Mills South
M4C - Woodbine Heights
M5C - St. James Town
M6C - Humewood-Cedarvale
M9C - Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
M1E - Guildwood,Morningside,West Hill
M4E - The Beaches
M5E - Berczy Park
M6E - Caledonia-Fairbanks
M1G - Woburn
M4G - Leaside
M5G - Central Bay Street
M6G - Christie
M1H - Cedarbrae
M2H - Hillcrest Village
M3H - Bathurst Manor,Downsview North,Wilson Heights
M4H - Thorncliffe Park
M5H - Adelaide,King,Richmond
M6H - Dovercourt Village,Dufferin
M1J - Scarborough Village
M2J - Fairview,Henry Farm,Oriole
M3J - Northwood Park,York Univers

Check size of the newly created dataframe.

In [16]:
toronto_venues.shape

(2219, 7)

In [17]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A - Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A - Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,M3A - Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A - Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A - Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


Let's check how many venues were returned for each neighborhood

In [18]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"M1B - Rouge,Malvern",2,2,2,2,2,2
"M1C - Highland Creek,Rouge Hill,Port Union",1,1,1,1,1,1
"M1E - Guildwood,Morningside,West Hill",7,7,7,7,7,7
M1G - Woburn,4,4,4,4,4,4
M1H - Cedarbrae,7,7,7,7,7,7
M1J - Scarborough Village,1,1,1,1,1,1
"M1K - East Birchmount Park,Ionview,Kennedy Park",5,5,5,5,5,5
"M1L - Clairlea,Golden Mile,Oakridge",9,9,9,9,9,9
"M1M - Cliffcrest,Cliffside,Scarborough Village West",2,2,2,2,2,2


Let's find out how many unique categories can be curated from all the returned venues

In [19]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 272 uniques categories.


## Analysing each Neighborhood

In [20]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True)

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M3A - Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [21]:
toronto_onehot.shape

(2219, 272)

#### Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [22]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"M1B - Rouge,Malvern",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

In [23]:
toronto_grouped.shape

(99, 272)

Printing each neighborhood along with the top 5 most common venues

In [24]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B - Rouge,Malvern----
                       venue  freq
0       Fast Food Restaurant   0.5
1                 Print Shop   0.5
2          Accessories Store   0.0
3  Middle Eastern Restaurant   0.0
4        Monument / Landmark   0.0


----M1C - Highland Creek,Rouge Hill,Port Union----
                 venue  freq
0                  Bar   1.0
1    Accessories Store   0.0
2   Miscellaneous Shop   0.0
3                Motel   0.0
4  Monument / Landmark   0.0


----M1E - Guildwood,Morningside,West Hill----
                 venue  freq
0          Pizza Place  0.14
1   Mexican Restaurant  0.14
2  Rental Car Location  0.14
3    Electronics Store  0.14
4                  Spa  0.14


----M1G - Woburn----
                venue  freq
0         Coffee Shop  0.50
1            Pharmacy  0.25
2   Korean Restaurant  0.25
3   Accessories Store  0.00
4  Miscellaneous Shop  0.00


----M1H - Cedarbrae----
                 venue  freq
0  Fried Chicken Joint  0.14
1               Bakery  0.14
2        

               venue  freq
0        Pizza Place  0.06
1        Coffee Shop  0.06
2            Brewery  0.06
3  Fish & Chips Shop  0.06
4       Burger Joint  0.06


----M4M - Studio District----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2            Gastropub  0.05
3  American Restaurant  0.05
4               Bakery  0.05


----M4N - Lawrence Park----
                venue  freq
0            Bus Line   0.2
1  Dim Sum Restaurant   0.2
2                Park   0.2
3         Swim School   0.2
4                Lake   0.2


----M4P - Davisville North----
            venue  freq
0            Park   0.1
1  Breakfast Spot   0.1
2           Hotel   0.1
3    Burger Joint   0.1
4             Gym   0.1


----M4R - North Toronto West----
                 venue  freq
0          Coffee Shop  0.12
1  Sporting Goods Shop  0.12
2       Clothing Store  0.12
3   Salon / Barbershop  0.06
4   Chinese Restaurant  0.06


----M4S - Davisville----
            venue  fr

4     Mobile Phone Shop  0.00


----M6P - High Park,The Junction South----
                 venue  freq
0   Mexican Restaurant  0.08
1                 Café  0.08
2        Grocery Store  0.04
3  Arts & Crafts Store  0.04
4       Sandwich Place  0.04


----M6R - Parkdale,Roncesvalles----
            venue  freq
0  Breakfast Spot  0.12
1       Gift Shop  0.12
2     Coffee Shop  0.06
3    Dessert Shop  0.06
4            Bank  0.06


----M6S - Runnymede,Swansea----
                venue  freq
0                Café  0.08
1         Pizza Place  0.08
2         Coffee Shop  0.08
3  Italian Restaurant  0.05
4    Sushi Restaurant  0.05


----M7A - Queen's Park----
                 venue  freq
0          Coffee Shop  0.26
1  Japanese Restaurant  0.05
2         Burger Joint  0.05
3                  Gym  0.05
4                Diner  0.05


----M7R - Canada Post Gateway Processing Centre----
                      venue  freq
0                     Hotel   0.2
1               Coffee Shop   0.2
2  Medit

## Place data in a dataframe

In [25]:
# Function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"M1B - Rouge,Malvern",Fast Food Restaurant,Print Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop,Dumpling Restaurant
1,"M1C - Highland Creek,Rouge Hill,Port Union",Bar,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant
2,"M1E - Guildwood,Morningside,West Hill",Medical Center,Spa,Pizza Place,Electronics Store,Breakfast Spot,Rental Car Location,Mexican Restaurant,Doner Restaurant,Diner,Discount Store
3,M1G - Woburn,Coffee Shop,Pharmacy,Korean Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
4,M1H - Cedarbrae,Hakka Restaurant,Athletics & Sports,Thai Restaurant,Bakery,Caribbean Restaurant,Bank,Fried Chicken Joint,Diner,Dog Run,Doner Restaurant


## Cluster Neighborhoods

Run *k*-means to cluster the neighborhoods into 8 clusters.

In [27]:
# set number of clusters
kclusters = 8

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)


# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 3, 5, 5, 5, 2, 5, 1, 5, 5])

In [28]:
toronto_grouped_clustering.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = dfPcode

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,M3A - Parkwoods,43.753259,-79.329656,1.0,Fast Food Restaurant,Food & Drink Shop,Park,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
1,North York,M4A - Victoria Village,43.725882,-79.315572,5.0,Coffee Shop,Portuguese Restaurant,Hockey Arena,Intersection,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,Downtown Toronto,"M5A - Harbourfront,Regent Park",43.654260,-79.360636,5.0,Coffee Shop,Café,Bakery,Park,Theater,Mexican Restaurant,Pub,Breakfast Spot,Brewery,Restaurant
3,North York,"M6A - Lawrence Heights,Lawrence Manor",43.718518,-79.464763,5.0,Clothing Store,Furniture / Home Store,Vietnamese Restaurant,Boutique,Miscellaneous Shop,Coffee Shop,Accessories Store,Event Space,Diner,Comfort Food Restaurant
4,Queen's Park,M7A - Queen's Park,43.662301,-79.389494,5.0,Coffee Shop,Japanese Restaurant,Gym,Burger Joint,Diner,Yoga Studio,Café,Italian Restaurant,Fast Food Restaurant,Seafood Restaurant


Remove any rows with NaN values 

In [30]:
toronto_merged.dropna(inplace=True)
toronto_merged["Cluster Labels"]= toronto_merged["Cluster Labels"].astype(int)
toronto_merged.head(10)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,M3A - Parkwoods,43.753259,-79.329656,1,Fast Food Restaurant,Food & Drink Shop,Park,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
1,North York,M4A - Victoria Village,43.725882,-79.315572,5,Coffee Shop,Portuguese Restaurant,Hockey Arena,Intersection,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,Downtown Toronto,"M5A - Harbourfront,Regent Park",43.654260,-79.360636,5,Coffee Shop,Café,Bakery,Park,Theater,Mexican Restaurant,Pub,Breakfast Spot,Brewery,Restaurant
3,North York,"M6A - Lawrence Heights,Lawrence Manor",43.718518,-79.464763,5,Clothing Store,Furniture / Home Store,Vietnamese Restaurant,Boutique,Miscellaneous Shop,Coffee Shop,Accessories Store,Event Space,Diner,Comfort Food Restaurant
4,Queen's Park,M7A - Queen's Park,43.662301,-79.389494,5,Coffee Shop,Japanese Restaurant,Gym,Burger Joint,Diner,Yoga Studio,Café,Italian Restaurant,Fast Food Restaurant,Seafood Restaurant
6,Scarborough,"M1B - Rouge,Malvern",43.806686,-79.194353,1,Fast Food Restaurant,Print Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop,Dumpling Restaurant
7,North York,M3B - Don Mills North,43.745906,-79.352188,5,Gym / Fitness Center,Japanese Restaurant,Caribbean Restaurant,Baseball Field,Café,Yoga Studio,Donut Shop,Discount Store,Dog Run,Doner Restaurant
8,East York,"M4B - Woodbine Gardens,Parkview Hill",43.706397,-79.309937,0,Pizza Place,Fast Food Restaurant,Rock Climbing Spot,Intersection,Breakfast Spot,Café,Athletics & Sports,Gastropub,Bank,Gym / Fitness Center
9,Downtown Toronto,"M5B - Ryerson,Garden District",43.657162,-79.378937,5,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant,Fast Food Restaurant,Japanese Restaurant,Bubble Tea Shop,Diner
10,North York,M6B - Glencairn,43.709577,-79.445073,5,Italian Restaurant,Bakery,Japanese Restaurant,Sushi Restaurant,Pub,Yoga Studio,Doner Restaurant,Discount Store,Dog Run,Donut Shop


In [31]:
toronto_merged.shape

(99, 15)

Visualise the clusters

In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine the clusters

Cluster 1

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"M4B - Woodbine Gardens,Parkview Hill",Pizza Place,Fast Food Restaurant,Rock Climbing Spot,Intersection,Breakfast Spot,Café,Athletics & Sports,Gastropub,Bank,Gym / Fitness Center
17,"M9C - Bloordale Gardens,Eringate,Markland Wood...",Pizza Place,Beer Store,Convenience Store,Café,Shopping Plaza,Liquor Store,Pharmacy,Gift Shop,Deli / Bodega,Eastern European Restaurant
29,M4H - Thorncliffe Park,Indian Restaurant,Yoga Studio,Bank,Gym,Intersection,Liquor Store,Park,Pharmacy,Pizza Place,Coffee Shop
46,M3L - Downsview West,Grocery Store,Shopping Mall,Bank,Hotel,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
50,M9L - Humber Summit,Pizza Place,Empanada Restaurant,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
60,M3N - Downsview Northwest,Gym / Fitness Center,Discount Store,Grocery Store,Athletics & Sports,Liquor Store,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop
63,"M6N - The Junction North,Runnymede",Pizza Place,Caribbean Restaurant,Grocery Store,Bus Line,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
70,M9P - Westmount,Pizza Place,Playground,Sandwich Place,Coffee Shop,Intersection,Chinese Restaurant,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner
72,M2R - Willowdale West,Pizza Place,Coffee Shop,Butcher,Pharmacy,Comfort Food Restaurant,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
82,"M1T - Clarks Corners,Sullivan,Tam O'Shanter",Pizza Place,Pharmacy,Noodle House,Fried Chicken Joint,Thai Restaurant,Rental Car Location,Chinese Restaurant,Italian Restaurant,Fast Food Restaurant,Donut Shop


Cluster 2

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A - Parkwoods,Fast Food Restaurant,Food & Drink Shop,Park,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
6,"M1B - Rouge,Malvern",Fast Food Restaurant,Print Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop,Dumpling Restaurant
14,M4C - Woodbine Heights,Skating Rink,Pharmacy,Beer Store,Curling Ice,Video Store,Cosmetics Shop,Spa,Park,Bus Stop,Empanada Restaurant
21,M6E - Caledonia-Fairbanks,Park,Pharmacy,Women's Store,Fast Food Restaurant,Market,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant
40,"M3K - CFB Toronto,Downsview East",Airport,Park,Bus Stop,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
44,"M1L - Clairlea,Golden Mile,Oakridge",Bus Line,Bakery,Soccer Field,Park,Fast Food Restaurant,Intersection,Metro Station,Diner,Discount Store,Dog Run
49,"M6L - Maple Leaf Park,North Park,Upwood Park",Basketball Court,Park,Bakery,Construction & Landscaping,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
61,M4N - Lawrence Park,Park,Bus Line,Swim School,Lake,Dim Sum Restaurant,Yoga Studio,Donut Shop,Discount Store,Dog Run,Doner Restaurant
62,M5N - Roselawn,Garden,Ice Cream Shop,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
68,"M5P - Forest Hill North,Forest Hill West",Park,Sushi Restaurant,Jewelry Store,Trail,Yoga Studio,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant


Cluster 3

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,M1J - Scarborough Village,Playground,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,College Rec Center
83,"M4T - Moore Park,Summerhill East",Playground,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,College Rec Center


Cluster 4

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,"M1C - Highland Creek,Rouge Hill,Port Union",Bar,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant


Cluster 5

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,"M9M - Emery,Humberlea",Baseball Field,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Fast Food Restaurant


Cluster 6

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4A - Victoria Village,Coffee Shop,Portuguese Restaurant,Hockey Arena,Intersection,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,"M5A - Harbourfront,Regent Park",Coffee Shop,Café,Bakery,Park,Theater,Mexican Restaurant,Pub,Breakfast Spot,Brewery,Restaurant
3,"M6A - Lawrence Heights,Lawrence Manor",Clothing Store,Furniture / Home Store,Vietnamese Restaurant,Boutique,Miscellaneous Shop,Coffee Shop,Accessories Store,Event Space,Diner,Comfort Food Restaurant
4,M7A - Queen's Park,Coffee Shop,Japanese Restaurant,Gym,Burger Joint,Diner,Yoga Studio,Café,Italian Restaurant,Fast Food Restaurant,Seafood Restaurant
7,M3B - Don Mills North,Gym / Fitness Center,Japanese Restaurant,Caribbean Restaurant,Baseball Field,Café,Yoga Studio,Donut Shop,Discount Store,Dog Run,Doner Restaurant
9,"M5B - Ryerson,Garden District",Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Italian Restaurant,Fast Food Restaurant,Japanese Restaurant,Bubble Tea Shop,Diner
10,M6B - Glencairn,Italian Restaurant,Bakery,Japanese Restaurant,Sushi Restaurant,Pub,Yoga Studio,Doner Restaurant,Discount Store,Dog Run,Donut Shop
13,"M3C - Flemingdon Park,Don Mills South",Gym,Asian Restaurant,Coffee Shop,Beer Store,Discount Store,Italian Restaurant,Dim Sum Restaurant,Restaurant,Sandwich Place,Chinese Restaurant
15,M5C - St. James Town,Coffee Shop,Restaurant,Café,Hotel,Clothing Store,Breakfast Spot,Gastropub,Bakery,Cosmetics Shop,American Restaurant
16,M6C - Humewood-Cedarvale,Field,Trail,Playground,Hockey Arena,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


Cluster 7

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,M4J - East Toronto,Park,Metro Station,Convenience Store,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
64,M9N - Weston,Park,Convenience Store,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
66,M2P - York Mills West,Park,Bank,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
77,"M9R - Kingsview Village,Martin Grove Gardens,R...",Park,Mobile Phone Shop,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
85,"M1V - Agincourt North,L'Amoreaux East,Milliken...",Playground,Park,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
91,M4W - Rosedale,Park,Trail,Playground,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


Cluster 8

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,"M9B - Cloverdale,Islington,Martin Grove,Prince...",Bank,Yoga Studio,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant


End of Assignment